In [ ]:
import pandas as pd

data = pd.read_csv('band_snapshots.csv')
print(data.columns)

# find the portion of collateral in crvUSD and the collateral token as a percentage
data['collateralPct'] = data['collateralUsd']/(data['collateralUsd']+data['stableCoin'])*100
data['stablecoinPct'] = 100-data['collateralPct']

# sort the data by user, marketId and timestamp
data = data.sort_values(by=['marketId', 'timestamp', 'index']).reset_index(drop=True)

In [ ]:
data['lossPctPerDay'] = 0

# count the days a user is in soft liquidation
data['softLiqDays'] = 0

# need to iterate through data to get the above data.  This is slow but works.
i = 0
dataActiveBands = data[(data['activeBand'] == data['index'])].copy()
length = len(dataActiveBands)
new_data = pd.DataFrame(columns=data.columns)

while i < length:

    # get the current row data
    row = dataActiveBands.iloc[i].copy()
    marketId = row['marketId']
    activeBand = int(row['activeBand'])
    timestamp = row['timestamp']
    
    collat_amount = (row['collateralUsd'] + row['stableCoin']) / row['oraclePrice']
    collat_value = row['collateralUsd'] + row['stableCoin']

    if i > 0 and marketId == prev_marketId:

        if activeBand >= prev_activeBand:
            ## price has dropped, calc loss in above bands
            bandRange = len(range(prev_activeBand, activeBand+1))
            for j in range(prev_activeBand, activeBand+1):
                print(j)
                prev_row = data[(data['marketId'] == marketId) & (data['timestamp'] == prev_timestamp) & (data['index'] == j)].copy()
                cur_row = data[(data['marketId'] == marketId) & (data['timestamp'] == timestamp) & (data['index'] == j)].copy()
                try:
                    prev_basePrice = prev_row['basePrice'].values[0]
                    basePrice = cur_row['basePrice'].values[0]
                except:
                    print("happened")
                    continue

                if prev_row['activeBand'].values[0] == j:
                    prev_price = prev_row['oraclePrice'].values[0]
                elif prev_row['activeBand'].values[0] < j:
                    prev_price = prev_basePrice * (99/100)**(j)
                else:
                    prev_price = prev_basePrice * (99/100)**(j+1)
                prev_collat_amount = (prev_row['collateralUsd'].values[0] + prev_row['stableCoin'].values[0]) / prev_price
                
                if cur_row['activeBand'].values[0] == j:
                    price = cur_row['oraclePrice'].values[0]
                elif cur_row['activeBand'].values[0] < j:
                    price = basePrice * (99/100)**(j)
                else:
                    price = basePrice * (99/100)**(j+1)

                collat_amount = (cur_row['collateralUsd'].values[0] + cur_row['stableCoin'].values[0]) / price
                prev_collat_amount = (prev_row['collateralUsd'].values[0] + prev_row['stableCoin'].values[0]) / prev_price
                lost_amount = prev_collat_amount - collat_amount

                time_days_diff = (timestamp - prev_timestamp) / bandRange / 86400
                
                # else we log the lost value and the time between snapshots
                lossPctPerDay = lost_amount / prev_collat_amount / time_days_diff
                if lossPctPerDay > 0 and lossPctPerDay < 0.10:
                    cur_row.loc[cur_row.index[0], 'softLiqDays'] = time_days_diff
                    cur_row.loc[cur_row.index[0], 'lossPctPerDay'] = lossPctPerDay
                    if not cur_row['marketId'].isnull().any():
                        print(f"market: {cur_row['marketId'].values[0]}")
                        new_data = pd.concat([new_data, cur_row], ignore_index=True)
                    else:
                        pass

    # set the previous values to the current values
    prev_collat_amount = collat_amount
    prev_marketId = marketId
    prev_timestamp = timestamp
    prev_activeBand = activeBand

    i += 1
    print(i)

In [ ]:
# get real soft liquidation subset of data
softLiqData = new_data.copy()
softLiqData['liquidity'] = softLiqData['collateralUsd'] + softLiqData['stableCoin']

# create bins for the number of bands a user chose
bins = [0, 1000, 5000, 20000, 100000, 500000, 2000000, 10000000]
labels = ['<1000', '1000-5000', '5000-20000', '20000-100000', '100000-500000', '500000-2000000', '2000000-10000000']
softLiqData.loc[:, 'liq_range'] = pd.cut(softLiqData['liquidity'], bins=bins, labels=labels, right=False)

# group the data by the number of bands a user chose
sl_liq_stats = softLiqData.groupby(['liq_range']).agg({
    'timestamp': 'count',
    'lossPctPerDay': ['min', 'median', 'mean', 'std', 'max'],
    'softLiqDays': 'sum'
}).reset_index(drop=False)

# rename the columns and save to csv
sl_liq_stats.columns = ['liq_range', 'entries', 'lossPctDay_min', 'lossPctDay_median', 'lossPctDay_mean', 'lossPctDay_std', 'lossPctDay_max', 'softLiqDays']
sl_liq_stats.to_csv('liquidity_soft_liq_stats.csv', index=False)
print(sl_liq_stats)